# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [43]:
!cat publications.tsv

7/24/11	High Performance Clustering for Web Person Name Disambiguation Using Topic Capturing	SIGIR2011 Workshop on Entity Orientation Search	We identify a topic drift problem in clustering web person name disambiguation and propose a fix by maintaining the topic words.	sigir2011namedisambiguation	"https://hunterhector.github.io/files/papers/Liu,_Lu,_Xu_-_2011_-_The_first_International_Workshop_on_Entity_Orientation_Search,_SIGIR2011_Workshop.pdf"	

## Import pandas

We are using the very handy pandas library for dataframes.

In [44]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [45]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,url_slug,paper_url,project_url
0,9/15/18,"Texar : A Modularized , Versatile , and Extens...",Proceedings of Workshop for NLP Open Source So...,"We introduce Texar, an open-source toolkit aim...",texar,https://hunterhector.github.io/files/papers/Hu...,https://github.com/asyml/texar
1,9/3/18,Automatic Event Salience Identification,EMNLP 2018,NaN,emnlp2018salience,https://hunterhector.github.io/files/papers/Li...,https://github.com/hunterhector/EventSalience
2,3/3/18,Towards Better Text Understanding and Retrieva...,SIGIR 2018,NaN,sigir2018rank_salience,https://hunterhector.github.io/files/papers/Xi...,http://boston.lti.cs.cmu.edu/appendices/SIGIR2...
3,1/13/18,Graph-Based Decoding for Event Sequencing and ...,7th International Conference on Computational ...,We study two types of relation: Event Corefere...,coling2018sequencing,https://hunterhector.github.io/files/papers/Li...,NaN
4,11/13/17,Multi-lingual Extraction and Integration of En...,TAC 2017,NaN,tac2017,https://hunterhector.github.io/files/papers/Ch...,NaN
5,11/13/17,"Events Detection, Coreference and Sequencing: ...",TAC 2017,NaN,tac2017,https://hunterhector.github.io/files/papers/Mi...,NaN
6,11/7/17,JointSem: Combining Query Entity Linking and E...,Proceedings of the 26th ACM International Conf...,"This work presents JointSem, a joint semantic ...",cikm2017joint,https://hunterhector.github.io/files/papers/Xi...,NaN
7,8/7/17,De-duping URLs with Sequence-to-Sequence Neura...,SIGIR 2017 Proceedings of the 40th Internation...,Traditional de-duping methods are usually limi...,sigir2017dedup,https://hunterhector.github.io/files/papers/Xu...,NaN
8,3/28/17,Overview of TAC-KBP 2016 Event Nugget Track,TAC 2016,This is an overview to the TAC KBP 2016 event ...,tac_2016overview,https://hunterhector.github.io/files/papers/Mi...,NaN
9,3/28/17,CMU-LTI at KBP 2016 Event Nugget Track,TAC 2016,This paper describes the LTI team's participat...,tac_2016event,https://hunterhector.github.io/files/papers/Li...,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [46]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [47]:
import os
for row, item in publications.iterrows():        
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
        
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"

# Too many URL and ugly    
#     if len(str(item.paper_url)) > 5:
#         md += "\npaperurl: '" + item.paper_url + "'"
    
#     md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    paper_url = '#'
    if len(str(item.paper_url)) > 5:
        paper_url = item.paper_url
        
    project_url = '#'
    if len(str(item.project_url)) > 5:
        project_url = item.project_url
        
    md += "\n[Paper](" + paper_url + ")"
    md += " \| "
    md += "[Project Page](" + project_url + ")\n"
        
#     md += "\nRecommended citation: " + item.citation
        
    md_filename = os.path.basename(md_filename)
           
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [48]:
!ls ../_publications/

11-sigir2011namedisambiguation.md 16-naacl2016coref.md
11-tac2011.md                     16-naacl_2016vpe.md
12-konvens2012.md                 17-cikm2017joint.md
12-lncs2012.md                    17-sigir2017dedup.md
12-semeval_2012.md                17-tac2017.md
12-www2012.md                     17-tac_2016event.md
14-lrec2014coref.md               17-tac_2016overview.md
14-lrec2014subevent.md            18-coling2018sequencing.md
14-ntcir11.md                     18-emnlp2018salience.md
15-naacl_event_workshop2015.md    18-sigir2018rank_salience.md
15-tac2015nugget.md               18-texar.md
15-tac2015overview.md             delete
16-acl_2016logic.md


In [49]:
!cat ../_publications/2018-03-03-sigir2018.md

cat: ../_publications/2018-03-03-sigir2018.md: No such file or directory
